In [1]:
import numpy as np
import pandas as pd
import regex as re
import random as rn
import ast
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,average_precision_score, precision_score,precision_recall_curve
from tqdm.notebook import tqdm
from tqdm import trange
import warnings
import pickle
import nltk
import math
import os
import json
import random
import re
import torch
import torch.nn as nn
import spacy 
from spacy import displacy
#import neptune.new as neptune                          
from torch.utils.data import (DataLoader, RandomSampler, WeightedRandomSampler, SequentialSampler, TensorDataset, Dataset)

In [2]:
import torch

# Confirm that the GPU is detected

assert torch.cuda.is_available()

# Get the GPU device name.
device_name = torch.cuda.get_device_name()
n_gpu = torch.cuda.device_count()
print(f"Found device: {device_name}, n_gpu: {n_gpu}")
device = torch.device("cuda")

Found device: Tesla P100-PCIE-16GB, n_gpu: 1


In [3]:
!pip install transformers
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
print('success!')

import os
import zipfile

# Download helper functions file
helper_file = drive.CreateFile({'id': '16HW-z9Y1tM3gZ_vFpJAuwUDohz91Aac-'})
helper_file.GetContentFile('helpers.py')
print('helper file downloaded! (helpers.py)')

# Download sample file of tweets
data_file = drive.CreateFile({'id': '1QcoAmjOYRtsMX7njjQTYooIbJHPc6Ese'})
data_file.GetContentFile('tweets.csv')
print('sample tweets downloaded! (tweets.csv)')

success!
helper file downloaded! (helpers.py)
sample tweets downloaded! (tweets.csv)


In [4]:
from transformers import RobertaTokenizer, RobertaForSequenceClassification
import torch
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=5)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.dense.bias', 'roberta.pooler.dense.weight', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.weight', 'classi

In [5]:
from google.colab import drive
drive.mount('/content/drive')
device = 'cuda' if torch.cuda.is_available() else 'cpu'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
inputs = tokenizer("Hello, my dog is cute haha", return_tensors="pt")
labels = torch.tensor([[1,2,3,4,5]], dtype=torch.float) # need dtype=float for BCEWithLogitsLoss
outputs = model(**inputs, labels=labels)
loss = outputs.loss
logits = outputs.logits

print(loss)
print(logits)

tensor(0.6255, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor([[-0.0205,  0.0296,  0.0752,  0.0796, -0.0358]],
       grad_fn=<AddmmBackward0>)


In [7]:
inputs['input_ids'].shape

torch.Size([1, 9])

In [8]:
labels = torch.tensor([[0,1,0,0,0]], dtype=torch.float)

In [9]:
model(inputs['input_ids'], token_type_ids=None, attention_mask=inputs['attention_mask'], labels=labels)

SequenceClassifierOutput([('loss',
                           tensor(0.7004, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)),
                          ('logits',
                           tensor([[-0.0205,  0.0296,  0.0752,  0.0796, -0.0358]],
                                  grad_fn=<AddmmBackward0>))])

In [10]:
model.classifier

RobertaClassificationHead(
  (dense): Linear(in_features=768, out_features=768, bias=True)
  (dropout): Dropout(p=0.1, inplace=False)
  (out_proj): Linear(in_features=768, out_features=5, bias=True)
)

In [11]:
tokenized = tokenizer.tokenize('I am watching an excellent movie')
print(tokenized)
print(tokenizer.encode(tokenized, add_special_tokens=False))
print(tokenizer.encode(tokenized, add_special_tokens=True))

['I', 'Ġam', 'Ġwatching', 'Ġan', 'Ġexcellent', 'Ġmovie']
[100, 524, 2494, 41, 4206, 1569]
[0, 100, 524, 2494, 41, 4206, 1569, 2]


## Data set

In [12]:
reviews_path = '/content/drive/MyDrive/gen_datasets/hotel_travel_reviews.json'
# reviews_path = '/content/drive/MyDrive/restaurant_reviews/res_reviews_final.json'
import json
f = open(reviews_path)
# data = json.load(f)
data = json.load(f)

In [13]:
reviews = list(data.keys())
labels = []
for j in reviews:
  labels.append(data[j])

In [14]:
tks = tokenizer(reviews, return_tensors="pt",padding=True, truncation=True,)
input_ids = tks['input_ids']
attention_masks = tks['attention_mask']

In [15]:
input_ids.shape

torch.Size([3552, 512])

In [16]:
# Convert the lists into tensors.
# input_ids = torch.cat(input_ids, dim=0)
# attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)
# Print sentence 0, now as a list of IDs.
print('Original: ', reviews[1])
print('Token IDs:', input_ids[1])

Original:  The food at Pop Century sucks! I can't believe the kind of food they have. It's disgusting and way over priced for the crap it is. I would have thought that it would be at least good food for being Disney. They over charge everything. You're practically forced to eat there since it's the only food they have there. The cups have a limit of refills and they time your drinks too. My first and last time staying at this resort. I do not recommend this place at all.
Token IDs: tensor([    0,   133,   689,    23,  7975,  9348, 29384,   328,    38,    64,
           75,   679,     5,   761,     9,   689,    51,    33,     4,    85,
           18, 21096,     8,   169,    81,  7663,    13,     5, 28342,    24,
           16,     4,    38,    74,    33,   802,    14,    24,    74,    28,
           23,   513,   205,   689,    13,   145,  4672,     4,   252,    81,
         1427,   960,     4,   370,   214, 15655,  1654,     7,  3529,    89,
          187,    24,    18,     5,   129,   

In [17]:
labels

tensor([2., 1., 4.,  ..., 1., 1., 1.])

In [20]:
labels.shape[0]
a = torch.zeros(labels.shape[0], 5)
idx = 0
for x in a:
  x[labels[idx].long()-1] = 1.0
  idx +=1


In [22]:
labels = a

## Create Train/Test/Valid splits

In [23]:
total = len(reviews)

num_train = int(total * .8)
num_val = int(total * .1)
num_test = total - num_train - num_val

# make lists of 3-tuples (already shuffled the dataframe in cell above)

train_set = [(input_ids[i], attention_masks[i], labels[i]) for i in range(num_train)]
val_set = [(input_ids[i], attention_masks[i], labels[i]) for i in range(num_train, num_val+num_train)]
test_set = [(input_ids[i], attention_masks[i], labels[i]) for i in range(num_val + num_train, total)]

train_text = [reviews[i] for i in range(num_train)]
val_text = [reviews[i] for i in range(num_train, num_val+num_train)]
test_text = [reviews[i] for i in range(num_val + num_train, total)]


## Training

In [24]:
from transformers import BertForSequenceClassification, AdamW, BertConfig

batch_size = 8
optimizer = AdamW(model.parameters(),
                  lr = 1e-6, # args.learning_rate - default is 5e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8
                )
epochs = 10
model.cuda()

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerN

In [41]:
import numpy as np
# function to get validation accuracy
def get_validation_performance(val_set):
    # Put the model in evaluation mode
    model.eval()

    # Tracking variables
    total_correct = 0
    correct_ = np.array([])
    # total_eval_accuracy = 0
    total_eval_loss = 0

    num_batches = int(len(val_set)/batch_size) + 1

    for i in range(num_batches):
      end_index = min(batch_size * (i+1), len(val_set))

      batch = val_set[i*batch_size:end_index]
      if len(batch) == 0: continue
      else:
        input_id_tensors = torch.stack([data[0] for data in batch])
        input_mask_tensors = torch.stack([data[1] for data in batch])
        label_tensors = torch.stack([data[2] for data in batch])
        
        # Move tensors to the GPU
        b_input_ids = input_id_tensors.to(device)
        b_input_mask = input_mask_tensors.to(device)
        b_labels = label_tensors.to(device)
          
        # Tell pytorch not to bother with constructing the compute graph during
        # the forward pass, since this is only needed for backprop (training).
        with torch.no_grad():        

          # Forward pass, calculate logit predictions.
          outputs = model(b_input_ids, 
                                  token_type_ids=None, 
                                  attention_mask=b_input_mask,
                                  labels=b_labels)
          loss = outputs.loss
          logits = outputs.logits
              
          # Accumulate the validation loss.
          total_eval_loss += loss.item()
          # Move logits and labels to CPU
          logits = logits.detach().cpu().numpy()
          label_ids = b_labels.to('cpu').numpy()
          # Calculate the number of correctly labeled examples in batch
          pred_flat = np.argmax(logits, axis=1).flatten()+1
          # labels_flat = label_ids.flatten()
          true_flat = np.argmax(label_ids, axis=1).flatten() +1
          labels_flat = true_flat
          correct_ = np.append(correct_,pred_flat == labels_flat)
          num_correct = np.sum(pred_flat == labels_flat)
          total_correct += num_correct
          # Report the final accuracy for this validation run.
    avg_val_accuracy = total_correct / len(val_set)
    return avg_val_accuracy, correct_

In [42]:
import random

# training loop

# For each epoch...
for epoch_i in range(0, epochs):
    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Reset the total loss for this epoch.
    total_train_loss = 0

    # Put the model into training mode.
    model.train()

    # For each batch of training data...
    num_batches = int(len(train_set)/batch_size) + 1

    for i in range(num_batches):
      end_index = min(batch_size * (i+1), len(train_set))

      batch = train_set[i*batch_size:end_index]
      if len(batch)==0: continue

      input_id_tensors = torch.stack([data[0] for data in batch])
      input_mask_tensors = torch.stack([data[1] for data in batch])
      label_tensors = torch.stack([data[2] for data in batch])

      # Move tensors to the GPU
      b_input_ids = input_id_tensors.to(device)
      b_input_mask = input_mask_tensors.to(device)
      b_labels = label_tensors.to(device)

      # Clear the previously calculated gradient
      model.zero_grad()

      # Perform a forward pass (evaluate the model on this training batch).
      # inputs = {}
      # inputs['input_ids'] = b_input_ids
      # inputs['attention_mask'] = b_input_mask
      # outputs = model(**inputs, labels=b_labels)

      # print(b_input_ids.shape)
      # print(b_input_mask.shape)
      # print(b_labels.shape)
      outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
      loss = outputs.loss
      logits = outputs.logits

      total_train_loss += loss.item()

      # Perform a backward pass to calculate the gradients.
      loss.backward()
      # Update parameters and take a step using the computed gradient.
      optimizer.step()
        
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set. Implement this function in the cell above.
    print(f"Total loss: {total_train_loss}")
    val_acc,_ = get_validation_performance(val_set)
    print(f"Validation accuracy: {val_acc}")
    
print("")
print("Training complete!")



======== Epoch 1 / 10 ========
Training...
Total loss: 74.42883708328009
Validation accuracy: 0.6028169014084507

======== Epoch 2 / 10 ========
Training...
Total loss: 70.8466989658773
Validation accuracy: 0.6112676056338028

======== Epoch 3 / 10 ========
Training...
Total loss: 67.06875151768327
Validation accuracy: 0.6028169014084507

======== Epoch 4 / 10 ========
Training...
Total loss: 63.810634423047304
Validation accuracy: 0.6028169014084507

======== Epoch 5 / 10 ========
Training...
Total loss: 60.22368446364999
Validation accuracy: 0.6056338028169014

======== Epoch 6 / 10 ========
Training...
Total loss: 56.86388675123453
Validation accuracy: 0.6

======== Epoch 7 / 10 ========
Training...
Total loss: 54.05408437177539
Validation accuracy: 0.6169014084507042

======== Epoch 8 / 10 ========
Training...
Total loss: 51.147180158644915
Validation accuracy: 0.6056338028169014

======== Epoch 9 / 10 ========
Training...
Total loss: 47.07723678648472
Validation accuracy: 0.60281

In [43]:
path = '/content/drive/MyDrive/Roberta-yelp-sentiment/'

In [44]:
model.save_pretrained(path)
tokenizer.save_pretrained(path)

('/content/drive/MyDrive/Roberta-yelp-sentiment/tokenizer_config.json',
 '/content/drive/MyDrive/Roberta-yelp-sentiment/special_tokens_map.json',
 '/content/drive/MyDrive/Roberta-yelp-sentiment/vocab.json',
 '/content/drive/MyDrive/Roberta-yelp-sentiment/merges.txt',
 '/content/drive/MyDrive/Roberta-yelp-sentiment/added_tokens.json')